In [161]:
import os
import json
import pandas as pd

# Define the path to the directory containing the json files
dir_path = './results'

# Initialize a list to hold the contents of all config.json files
config_contents = []

# Walk through the directory
for root, dirs, files in os.walk(dir_path):
    for file in files:
        # Check if the current file is a config.json
        if file == 'config.json':
            # Construct the full path to the file
            file_path = os.path.join(root, file)
            # Open and read the config.json file
            with open(file_path, 'r') as json_file:
                config_data = json.load(json_file)
                # Add the file name to the dictionary
                config_data['file_name'] = file_path.split("/")[-2]
                # Append the data to the list
                config_contents.append(config_data)

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(config_contents)

# Split the resize into two separate columns for height and width
df['resize_height'], df['resize_width'] = zip(*df['resize'])
df.drop('resize', axis=1, inplace=True)  # drop the original resize column
df.drop('data_dir', axis=1, inplace=True)
df.drop('model_dir', axis=1, inplace=True)

df

,seed,epochs,dataset,augmentation,dataloader,batch_size,valid_batch_size,model,optimizer,lr,...,file_name,kfold,cutmix,scheduler,lr_decay_rate,patience,early_stopping,mixup,resize_height,resize_width
0,42,30,MixUpSplitByProfileDataset,BaseAugmentation,MaskDataLoader,512,1000,SwinTransformerBase224V1,Adam,0.001,...,SwinTransformerBase224V1_Focal_W1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,224,224
1,42,15,MaskSplitByProfileDataset,CustomAugmentation,MaskDataLoader,256,45,SwinTransformerBase224V1,Adam,0.001,...,test_SwinTransformerV1_f1_fold0,1.0,0.0,StepLR,0.5,5.0,5.0,NaN,224,224
2,42,30,MaskSplitByProfileDataset,BaseAugmentation,MaskDataLoader,512,1000,EfficientNetB0MultiHead,Adam,0.001,...,EfficientNetB0MultiHead_MixUp_1,NaN,NaN,NaN,NaN,NaN,NaN,1.0,224,224
3,42,30,MaskSplitByProfileDataset,CustomAugmentation,MaskDataLoader,256,90,SwinTransformerBase224V1,Adam,0.001,...,final_SwinTransformerV1_f1_fold1,1.0,0.0,StepLR,0.5,5.0,5.0,NaN,224,224
4,42,30,MaskSplitByProfileDataset,BaseAugmentation,MaskDataLoader,512,1000,EfficientViTB3MultiHead,Adam,0.001,...,EfficientViTB3MultiHeadV2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,128,96
5,42,100,MaskSplitByProfileDataset,CustomAugmentation,MaskDataLoader,256,90,SwinTransformerBase224V1,Adam,0.001,...,finale_SwinTransformerV1,0.0,0.0,StepLR,0.5,5.0,7.0,NaN,224,224
6,42,30,MaskSplitByProfileDataset,BaseAugmentation,MaskDataLoader,512,1000,SwinTransformerBase224V1,Adam,0.001,...,fix_SwinTransformerBase224V1_focal_W1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,224,224
7,42,15,MaskSplitByProfileDataset,CustomAugmentation,MaskDataLoader,256,45,SwinTransformerBase224V1,Adam,0.001,...,test_SwinTransformerV1_f1_fold2,1.0,0.0,StepLR,0.5,5.0,5.0,NaN,224,224
8,42,30,MaskSplitByProfileDataset,BaseAugmentation,MaskDataLoader,512,1000,FocalNetTinySRFMultiHead,Adam,0.001,...,FocalNetTinySRFMultiHeadV2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,128,96
9,42,30,MaskSplitByProfileDataset,CustomAugmentation,MaskDataLoader,256,1000,SwinTransformerBase224V1,Adam,0.001,...,fix_SwinTransformerV1_f1,NaN,NaN,NaN,NaN,NaN,NaN,0.0,224,224


### 결측치 처리

In [162]:
df.isna().sum()

seed                 0
epochs               0
dataset              0
augmentation         0
dataloader           0
batch_size           0
valid_batch_size     0
model                0
optimizer            0
lr                   0
val_ratio            0
criterion            0
weight               7
lr_decay_step        0
log_interval         0
best_model           5
name                 0
wandb                0
save_val_table       2
multi_head           0
use_caution_data     0
file_name            0
kfold               13
cutmix              13
scheduler           13
lr_decay_rate       13
patience            13
early_stopping      13
mixup               21
resize_height        0
resize_width         0
dtype: int64

1. weight -> SwinTransformerBase224V1_focal_W2 = "min"으로 변경하고 나머지는 "max"로 통일
2. save_val_table ->

In [163]:
df.columns

Index(['seed', 'epochs', 'dataset', 'augmentation', 'dataloader', 'batch_size',
       'valid_batch_size', 'model', 'optimizer', 'lr', 'val_ratio',
       'criterion', 'weight', 'lr_decay_step', 'log_interval', 'best_model',
       'name', 'wandb', 'save_val_table', 'multi_head', 'use_caution_data',
       'file_name', 'kfold', 'cutmix', 'scheduler', 'lr_decay_rate',
       'patience', 'early_stopping', 'mixup', 'resize_height', 'resize_width'],
      dtype='object')

#### weight

In [164]:
df[df["weight"].isna() == True]

,seed,epochs,dataset,augmentation,dataloader,batch_size,valid_batch_size,model,optimizer,lr,...,file_name,kfold,cutmix,scheduler,lr_decay_rate,patience,early_stopping,mixup,resize_height,resize_width
4,42,30,MaskSplitByProfileDataset,BaseAugmentation,MaskDataLoader,512,1000,EfficientViTB3MultiHead,Adam,0.001,...,EfficientViTB3MultiHeadV2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,128,96
6,42,30,MaskSplitByProfileDataset,BaseAugmentation,MaskDataLoader,512,1000,SwinTransformerBase224V1,Adam,0.001,...,fix_SwinTransformerBase224V1_focal_W1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,224,224
8,42,30,MaskSplitByProfileDataset,BaseAugmentation,MaskDataLoader,512,1000,FocalNetTinySRFMultiHead,Adam,0.001,...,FocalNetTinySRFMultiHeadV2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,128,96
10,42,30,MaskSplitByProfileDataset,BaseAugmentation,MaskDataLoader,512,1000,SwinTransformerBase224V1,Adam,0.001,...,SwinTransformerBase224V1_focal_W2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,224,224
11,42,30,MaskSplitByProfileDataset,BaseAugmentation,MaskDataLoader,512,1000,SwinTransformerBase224V1,Adam,0.001,...,SwinTransformerBase224_best_loss,NaN,NaN,NaN,NaN,NaN,NaN,NaN,224,224
16,42,30,MaskSplitByProfileDataset,BaseAugmentation,MaskDataLoader,512,1000,SwinTransformerBase224,Adam,0.001,...,fix_SwinTransformerBase224,NaN,NaN,NaN,NaN,NaN,NaN,NaN,224,224
19,42,30,MaskSplitByProfileDataset,BaseAugmentation,MaskDataLoader,512,64,SwinTransformerBase224,Adam,0.001,...,SwinTransformerBase224_focal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,224,224


- 6번 - 실제로 max를 사용했으나 argument로 전달 이전 시점이기 때문에 "max"로 변경
- 8번 - 실제로 min를 사용했으나 argument로 전달 이전 시점이기 때문에 "min"로 변경
- 19번 - weight 값을 주기 이전 loss function만 focal을 사용 했기 때문에 "none"으로 변경
- 나머지 - 사용한 loss function이 f1 이기 때문에 전부 "none"으로 변경

In [165]:
df[df["weight"].isna() == True].criterion

4        f1
6     focal
8        f1
10    focal
11       f1
16       f1
19    focal
Name: criterion, dtype: object

In [166]:
df.loc[6, "weight"] = "max"
df[df["file_name"] == "fix_SwinTransformerBase224V1_focal_W1"].weight

6    max
Name: weight, dtype: object

In [167]:
df.loc[10, "weight"] = "min"
df[df["file_name"] == "SwinTransformerBase224V1_focal_W2"].weight

10    min
Name: weight, dtype: object

In [168]:
rows = df[df["weight"].isna() == True].index.values
df.loc[rows, "weight"] = "none"
df.loc[rows, "weight"]

4     none
8     none
11    none
16    none
19    none
Name: weight, dtype: object

#### best model

best model 기준을 loss로 지정한 모델은 11번 인덱스 하나이므로 11번 인덱스를 제외한 모든 인덱스는 acc로 변경

In [169]:
df[df["best_model"].isna() == True]

,seed,epochs,dataset,augmentation,dataloader,batch_size,valid_batch_size,model,optimizer,lr,...,file_name,kfold,cutmix,scheduler,lr_decay_rate,patience,early_stopping,mixup,resize_height,resize_width
4,42,30,MaskSplitByProfileDataset,BaseAugmentation,MaskDataLoader,512,1000,EfficientViTB3MultiHead,Adam,0.001,...,EfficientViTB3MultiHeadV2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,128,96
8,42,30,MaskSplitByProfileDataset,BaseAugmentation,MaskDataLoader,512,1000,FocalNetTinySRFMultiHead,Adam,0.001,...,FocalNetTinySRFMultiHeadV2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,128,96
11,42,30,MaskSplitByProfileDataset,BaseAugmentation,MaskDataLoader,512,1000,SwinTransformerBase224V1,Adam,0.001,...,SwinTransformerBase224_best_loss,NaN,NaN,NaN,NaN,NaN,NaN,NaN,224,224
16,42,30,MaskSplitByProfileDataset,BaseAugmentation,MaskDataLoader,512,1000,SwinTransformerBase224,Adam,0.001,...,fix_SwinTransformerBase224,NaN,NaN,NaN,NaN,NaN,NaN,NaN,224,224
19,42,30,MaskSplitByProfileDataset,BaseAugmentation,MaskDataLoader,512,64,SwinTransformerBase224,Adam,0.001,...,SwinTransformerBase224_focal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,224,224


In [170]:
df.loc[11, "file_name"]

'SwinTransformerBase224_best_loss'

In [171]:
df.loc[11, "best_model"] = "loss"
df.loc[11, "best_model"]

'loss'

In [172]:
df[df["best_model"].isna() == True]

,seed,epochs,dataset,augmentation,dataloader,batch_size,valid_batch_size,model,optimizer,lr,...,file_name,kfold,cutmix,scheduler,lr_decay_rate,patience,early_stopping,mixup,resize_height,resize_width
4,42,30,MaskSplitByProfileDataset,BaseAugmentation,MaskDataLoader,512,1000,EfficientViTB3MultiHead,Adam,0.001,...,EfficientViTB3MultiHeadV2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,128,96
8,42,30,MaskSplitByProfileDataset,BaseAugmentation,MaskDataLoader,512,1000,FocalNetTinySRFMultiHead,Adam,0.001,...,FocalNetTinySRFMultiHeadV2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,128,96
16,42,30,MaskSplitByProfileDataset,BaseAugmentation,MaskDataLoader,512,1000,SwinTransformerBase224,Adam,0.001,...,fix_SwinTransformerBase224,NaN,NaN,NaN,NaN,NaN,NaN,NaN,224,224
19,42,30,MaskSplitByProfileDataset,BaseAugmentation,MaskDataLoader,512,64,SwinTransformerBase224,Adam,0.001,...,SwinTransformerBase224_focal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,224,224


In [174]:
rows = df[df["best_model"].isna() == True].index.values
df.loc[rows, "best_model"] = "acc"
df.loc[rows, "best_model"]

4     acc
8     acc
16    acc
19    acc
Name: best_model, dtype: object

#### save_val_table

전부 테이블을 사용안 한 것으로 처리 -> 0으로 변경

In [175]:
df[df["save_val_table"].isna() == True]

,seed,epochs,dataset,augmentation,dataloader,batch_size,valid_batch_size,model,optimizer,lr,...,file_name,kfold,cutmix,scheduler,lr_decay_rate,patience,early_stopping,mixup,resize_height,resize_width
4,42,30,MaskSplitByProfileDataset,BaseAugmentation,MaskDataLoader,512,1000,EfficientViTB3MultiHead,Adam,0.001,...,EfficientViTB3MultiHeadV2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,128,96
8,42,30,MaskSplitByProfileDataset,BaseAugmentation,MaskDataLoader,512,1000,FocalNetTinySRFMultiHead,Adam,0.001,...,FocalNetTinySRFMultiHeadV2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,128,96


In [176]:
df.loc[[4, 8], "save_val_table"] = 0
df.loc[[4, 8]].save_val_table

4    0.0
8    0.0
Name: save_val_table, dtype: float64

#### k-fold

전부 사용 안 한 것으로 처리 -> 0으로 변경

In [177]:
df[df["kfold"].isna() == True]

,seed,epochs,dataset,augmentation,dataloader,batch_size,valid_batch_size,model,optimizer,lr,...,file_name,kfold,cutmix,scheduler,lr_decay_rate,patience,early_stopping,mixup,resize_height,resize_width
0,42,30,MixUpSplitByProfileDataset,BaseAugmentation,MaskDataLoader,512,1000,SwinTransformerBase224V1,Adam,0.001,...,SwinTransformerBase224V1_Focal_W1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,224,224
2,42,30,MaskSplitByProfileDataset,BaseAugmentation,MaskDataLoader,512,1000,EfficientNetB0MultiHead,Adam,0.001,...,EfficientNetB0MultiHead_MixUp_1,NaN,NaN,NaN,NaN,NaN,NaN,1.0,224,224
4,42,30,MaskSplitByProfileDataset,BaseAugmentation,MaskDataLoader,512,1000,EfficientViTB3MultiHead,Adam,0.001,...,EfficientViTB3MultiHeadV2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,128,96
6,42,30,MaskSplitByProfileDataset,BaseAugmentation,MaskDataLoader,512,1000,SwinTransformerBase224V1,Adam,0.001,...,fix_SwinTransformerBase224V1_focal_W1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,224,224
8,42,30,MaskSplitByProfileDataset,BaseAugmentation,MaskDataLoader,512,1000,FocalNetTinySRFMultiHead,Adam,0.001,...,FocalNetTinySRFMultiHeadV2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,128,96
9,42,30,MaskSplitByProfileDataset,CustomAugmentation,MaskDataLoader,256,1000,SwinTransformerBase224V1,Adam,0.001,...,fix_SwinTransformerV1_f1,NaN,NaN,NaN,NaN,NaN,NaN,0.0,224,224
10,42,30,MaskSplitByProfileDataset,BaseAugmentation,MaskDataLoader,512,1000,SwinTransformerBase224V1,Adam,0.001,...,SwinTransformerBase224V1_focal_W2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,224,224
11,42,30,MaskSplitByProfileDataset,BaseAugmentation,MaskDataLoader,512,1000,SwinTransformerBase224V1,Adam,0.001,...,SwinTransformerBase224_best_loss,NaN,NaN,NaN,NaN,NaN,NaN,NaN,224,224
13,42,30,MaskSplitByProfileDataset,BaseAugmentation,MaskDataLoader,512,1000,SwinTransformerBase224V3,Adam,0.001,...,SwinTransformerV3_3stage,NaN,NaN,NaN,NaN,NaN,NaN,0.0,224,224
14,42,30,MaskSplitByProfileDataset,BaseAugmentation,MaskDataLoader,512,1000,SwinTransformerBase224V2,Adam,0.001,...,SwinTransformerV2_2stage,NaN,NaN,NaN,NaN,NaN,NaN,0.0,224,224


In [178]:
rows = df[df["kfold"].isna() == True].index.values
df.loc[rows, "kfold"] = 0
df.loc[rows, "kfold"]

0     0.0
2     0.0
4     0.0
6     0.0
8     0.0
9     0.0
10    0.0
11    0.0
13    0.0
14    0.0
16    0.0
19    0.0
22    0.0
Name: kfold, dtype: float64

#### cutmix

전부 사용 안 한 것으로 처리 -> 0으로 변경

In [179]:
df[df["cutmix"].isna() == True]

,seed,epochs,dataset,augmentation,dataloader,batch_size,valid_batch_size,model,optimizer,lr,...,file_name,kfold,cutmix,scheduler,lr_decay_rate,patience,early_stopping,mixup,resize_height,resize_width
0,42,30,MixUpSplitByProfileDataset,BaseAugmentation,MaskDataLoader,512,1000,SwinTransformerBase224V1,Adam,0.001,...,SwinTransformerBase224V1_Focal_W1,0.0,NaN,NaN,NaN,NaN,NaN,NaN,224,224
2,42,30,MaskSplitByProfileDataset,BaseAugmentation,MaskDataLoader,512,1000,EfficientNetB0MultiHead,Adam,0.001,...,EfficientNetB0MultiHead_MixUp_1,0.0,NaN,NaN,NaN,NaN,NaN,1.0,224,224
4,42,30,MaskSplitByProfileDataset,BaseAugmentation,MaskDataLoader,512,1000,EfficientViTB3MultiHead,Adam,0.001,...,EfficientViTB3MultiHeadV2,0.0,NaN,NaN,NaN,NaN,NaN,NaN,128,96
6,42,30,MaskSplitByProfileDataset,BaseAugmentation,MaskDataLoader,512,1000,SwinTransformerBase224V1,Adam,0.001,...,fix_SwinTransformerBase224V1_focal_W1,0.0,NaN,NaN,NaN,NaN,NaN,NaN,224,224
8,42,30,MaskSplitByProfileDataset,BaseAugmentation,MaskDataLoader,512,1000,FocalNetTinySRFMultiHead,Adam,0.001,...,FocalNetTinySRFMultiHeadV2,0.0,NaN,NaN,NaN,NaN,NaN,NaN,128,96
9,42,30,MaskSplitByProfileDataset,CustomAugmentation,MaskDataLoader,256,1000,SwinTransformerBase224V1,Adam,0.001,...,fix_SwinTransformerV1_f1,0.0,NaN,NaN,NaN,NaN,NaN,0.0,224,224
10,42,30,MaskSplitByProfileDataset,BaseAugmentation,MaskDataLoader,512,1000,SwinTransformerBase224V1,Adam,0.001,...,SwinTransformerBase224V1_focal_W2,0.0,NaN,NaN,NaN,NaN,NaN,NaN,224,224
11,42,30,MaskSplitByProfileDataset,BaseAugmentation,MaskDataLoader,512,1000,SwinTransformerBase224V1,Adam,0.001,...,SwinTransformerBase224_best_loss,0.0,NaN,NaN,NaN,NaN,NaN,NaN,224,224
13,42,30,MaskSplitByProfileDataset,BaseAugmentation,MaskDataLoader,512,1000,SwinTransformerBase224V3,Adam,0.001,...,SwinTransformerV3_3stage,0.0,NaN,NaN,NaN,NaN,NaN,0.0,224,224
14,42,30,MaskSplitByProfileDataset,BaseAugmentation,MaskDataLoader,512,1000,SwinTransformerBase224V2,Adam,0.001,...,SwinTransformerV2_2stage,0.0,NaN,NaN,NaN,NaN,NaN,0.0,224,224


In [180]:
rows = df[df["cutmix"].isna() == True].index.values
df.loc[rows, "cutmix"] = 0
df.loc[rows, "cutmix"]

0     0.0
2     0.0
4     0.0
6     0.0
8     0.0
9     0.0
10    0.0
11    0.0
13    0.0
14    0.0
16    0.0
19    0.0
22    0.0
Name: cutmix, dtype: float64

#### scheduler

전부 StepLR 로 변경

In [181]:
df[df["scheduler"].isna() == True]

,seed,epochs,dataset,augmentation,dataloader,batch_size,valid_batch_size,model,optimizer,lr,...,file_name,kfold,cutmix,scheduler,lr_decay_rate,patience,early_stopping,mixup,resize_height,resize_width
0,42,30,MixUpSplitByProfileDataset,BaseAugmentation,MaskDataLoader,512,1000,SwinTransformerBase224V1,Adam,0.001,...,SwinTransformerBase224V1_Focal_W1,0.0,0.0,NaN,NaN,NaN,NaN,NaN,224,224
2,42,30,MaskSplitByProfileDataset,BaseAugmentation,MaskDataLoader,512,1000,EfficientNetB0MultiHead,Adam,0.001,...,EfficientNetB0MultiHead_MixUp_1,0.0,0.0,NaN,NaN,NaN,NaN,1.0,224,224
4,42,30,MaskSplitByProfileDataset,BaseAugmentation,MaskDataLoader,512,1000,EfficientViTB3MultiHead,Adam,0.001,...,EfficientViTB3MultiHeadV2,0.0,0.0,NaN,NaN,NaN,NaN,NaN,128,96
6,42,30,MaskSplitByProfileDataset,BaseAugmentation,MaskDataLoader,512,1000,SwinTransformerBase224V1,Adam,0.001,...,fix_SwinTransformerBase224V1_focal_W1,0.0,0.0,NaN,NaN,NaN,NaN,NaN,224,224
8,42,30,MaskSplitByProfileDataset,BaseAugmentation,MaskDataLoader,512,1000,FocalNetTinySRFMultiHead,Adam,0.001,...,FocalNetTinySRFMultiHeadV2,0.0,0.0,NaN,NaN,NaN,NaN,NaN,128,96
9,42,30,MaskSplitByProfileDataset,CustomAugmentation,MaskDataLoader,256,1000,SwinTransformerBase224V1,Adam,0.001,...,fix_SwinTransformerV1_f1,0.0,0.0,NaN,NaN,NaN,NaN,0.0,224,224
10,42,30,MaskSplitByProfileDataset,BaseAugmentation,MaskDataLoader,512,1000,SwinTransformerBase224V1,Adam,0.001,...,SwinTransformerBase224V1_focal_W2,0.0,0.0,NaN,NaN,NaN,NaN,NaN,224,224
11,42,30,MaskSplitByProfileDataset,BaseAugmentation,MaskDataLoader,512,1000,SwinTransformerBase224V1,Adam,0.001,...,SwinTransformerBase224_best_loss,0.0,0.0,NaN,NaN,NaN,NaN,NaN,224,224
13,42,30,MaskSplitByProfileDataset,BaseAugmentation,MaskDataLoader,512,1000,SwinTransformerBase224V3,Adam,0.001,...,SwinTransformerV3_3stage,0.0,0.0,NaN,NaN,NaN,NaN,0.0,224,224
14,42,30,MaskSplitByProfileDataset,BaseAugmentation,MaskDataLoader,512,1000,SwinTransformerBase224V2,Adam,0.001,...,SwinTransformerV2_2stage,0.0,0.0,NaN,NaN,NaN,NaN,0.0,224,224


In [182]:
rows = df[df["scheduler"].isna() == True].index.values
df.loc[rows, "scheduler"] = "StepLR"
df.loc[rows, "scheduler"]

0     StepLR
2     StepLR
4     StepLR
6     StepLR
8     StepLR
9     StepLR
10    StepLR
11    StepLR
13    StepLR
14    StepLR
16    StepLR
19    StepLR
22    StepLR
Name: scheduler, dtype: object

#### lr_decay_rate

전부 0.5로 변경

In [183]:
df[df["lr_decay_rate"].isna() == True]

,seed,epochs,dataset,augmentation,dataloader,batch_size,valid_batch_size,model,optimizer,lr,...,file_name,kfold,cutmix,scheduler,lr_decay_rate,patience,early_stopping,mixup,resize_height,resize_width
0,42,30,MixUpSplitByProfileDataset,BaseAugmentation,MaskDataLoader,512,1000,SwinTransformerBase224V1,Adam,0.001,...,SwinTransformerBase224V1_Focal_W1,0.0,0.0,StepLR,NaN,NaN,NaN,NaN,224,224
2,42,30,MaskSplitByProfileDataset,BaseAugmentation,MaskDataLoader,512,1000,EfficientNetB0MultiHead,Adam,0.001,...,EfficientNetB0MultiHead_MixUp_1,0.0,0.0,StepLR,NaN,NaN,NaN,1.0,224,224
4,42,30,MaskSplitByProfileDataset,BaseAugmentation,MaskDataLoader,512,1000,EfficientViTB3MultiHead,Adam,0.001,...,EfficientViTB3MultiHeadV2,0.0,0.0,StepLR,NaN,NaN,NaN,NaN,128,96
6,42,30,MaskSplitByProfileDataset,BaseAugmentation,MaskDataLoader,512,1000,SwinTransformerBase224V1,Adam,0.001,...,fix_SwinTransformerBase224V1_focal_W1,0.0,0.0,StepLR,NaN,NaN,NaN,NaN,224,224
8,42,30,MaskSplitByProfileDataset,BaseAugmentation,MaskDataLoader,512,1000,FocalNetTinySRFMultiHead,Adam,0.001,...,FocalNetTinySRFMultiHeadV2,0.0,0.0,StepLR,NaN,NaN,NaN,NaN,128,96
9,42,30,MaskSplitByProfileDataset,CustomAugmentation,MaskDataLoader,256,1000,SwinTransformerBase224V1,Adam,0.001,...,fix_SwinTransformerV1_f1,0.0,0.0,StepLR,NaN,NaN,NaN,0.0,224,224
10,42,30,MaskSplitByProfileDataset,BaseAugmentation,MaskDataLoader,512,1000,SwinTransformerBase224V1,Adam,0.001,...,SwinTransformerBase224V1_focal_W2,0.0,0.0,StepLR,NaN,NaN,NaN,NaN,224,224
11,42,30,MaskSplitByProfileDataset,BaseAugmentation,MaskDataLoader,512,1000,SwinTransformerBase224V1,Adam,0.001,...,SwinTransformerBase224_best_loss,0.0,0.0,StepLR,NaN,NaN,NaN,NaN,224,224
13,42,30,MaskSplitByProfileDataset,BaseAugmentation,MaskDataLoader,512,1000,SwinTransformerBase224V3,Adam,0.001,...,SwinTransformerV3_3stage,0.0,0.0,StepLR,NaN,NaN,NaN,0.0,224,224
14,42,30,MaskSplitByProfileDataset,BaseAugmentation,MaskDataLoader,512,1000,SwinTransformerBase224V2,Adam,0.001,...,SwinTransformerV2_2stage,0.0,0.0,StepLR,NaN,NaN,NaN,0.0,224,224


In [184]:
rows = df[df["lr_decay_rate"].isna() == True].index.values
df.loc[rows, "lr_decay_rate"] = 0.5
df.loc[rows, "lr_decay_rate"]

0     0.5
2     0.5
4     0.5
6     0.5
8     0.5
9     0.5
10    0.5
11    0.5
13    0.5
14    0.5
16    0.5
19    0.5
22    0.5
Name: lr_decay_rate, dtype: float64

#### patience

scheduler를 StepLR만 사용 했으므로 default값인 5로 변경

In [185]:
df[df["patience"].isna() == True]

,seed,epochs,dataset,augmentation,dataloader,batch_size,valid_batch_size,model,optimizer,lr,...,file_name,kfold,cutmix,scheduler,lr_decay_rate,patience,early_stopping,mixup,resize_height,resize_width
0,42,30,MixUpSplitByProfileDataset,BaseAugmentation,MaskDataLoader,512,1000,SwinTransformerBase224V1,Adam,0.001,...,SwinTransformerBase224V1_Focal_W1,0.0,0.0,StepLR,0.5,NaN,NaN,NaN,224,224
2,42,30,MaskSplitByProfileDataset,BaseAugmentation,MaskDataLoader,512,1000,EfficientNetB0MultiHead,Adam,0.001,...,EfficientNetB0MultiHead_MixUp_1,0.0,0.0,StepLR,0.5,NaN,NaN,1.0,224,224
4,42,30,MaskSplitByProfileDataset,BaseAugmentation,MaskDataLoader,512,1000,EfficientViTB3MultiHead,Adam,0.001,...,EfficientViTB3MultiHeadV2,0.0,0.0,StepLR,0.5,NaN,NaN,NaN,128,96
6,42,30,MaskSplitByProfileDataset,BaseAugmentation,MaskDataLoader,512,1000,SwinTransformerBase224V1,Adam,0.001,...,fix_SwinTransformerBase224V1_focal_W1,0.0,0.0,StepLR,0.5,NaN,NaN,NaN,224,224
8,42,30,MaskSplitByProfileDataset,BaseAugmentation,MaskDataLoader,512,1000,FocalNetTinySRFMultiHead,Adam,0.001,...,FocalNetTinySRFMultiHeadV2,0.0,0.0,StepLR,0.5,NaN,NaN,NaN,128,96
9,42,30,MaskSplitByProfileDataset,CustomAugmentation,MaskDataLoader,256,1000,SwinTransformerBase224V1,Adam,0.001,...,fix_SwinTransformerV1_f1,0.0,0.0,StepLR,0.5,NaN,NaN,0.0,224,224
10,42,30,MaskSplitByProfileDataset,BaseAugmentation,MaskDataLoader,512,1000,SwinTransformerBase224V1,Adam,0.001,...,SwinTransformerBase224V1_focal_W2,0.0,0.0,StepLR,0.5,NaN,NaN,NaN,224,224
11,42,30,MaskSplitByProfileDataset,BaseAugmentation,MaskDataLoader,512,1000,SwinTransformerBase224V1,Adam,0.001,...,SwinTransformerBase224_best_loss,0.0,0.0,StepLR,0.5,NaN,NaN,NaN,224,224
13,42,30,MaskSplitByProfileDataset,BaseAugmentation,MaskDataLoader,512,1000,SwinTransformerBase224V3,Adam,0.001,...,SwinTransformerV3_3stage,0.0,0.0,StepLR,0.5,NaN,NaN,0.0,224,224
14,42,30,MaskSplitByProfileDataset,BaseAugmentation,MaskDataLoader,512,1000,SwinTransformerBase224V2,Adam,0.001,...,SwinTransformerV2_2stage,0.0,0.0,StepLR,0.5,NaN,NaN,0.0,224,224


In [186]:
rows = df[df["patience"].isna() == True].index.values
df.loc[rows, "patience"] = 5
df.loc[rows, "patience"]

0     5.0
2     5.0
4     5.0
6     5.0
8     5.0
9     5.0
10    5.0
11    5.0
13    5.0
14    5.0
16    5.0
19    5.0
22    5.0
Name: patience, dtype: float64

#### early_stopping

전부 사용안 한 것으로 처리 -> 0으로 변경

In [187]:
df[df["early_stopping"].isna() == True]

,seed,epochs,dataset,augmentation,dataloader,batch_size,valid_batch_size,model,optimizer,lr,...,file_name,kfold,cutmix,scheduler,lr_decay_rate,patience,early_stopping,mixup,resize_height,resize_width
0,42,30,MixUpSplitByProfileDataset,BaseAugmentation,MaskDataLoader,512,1000,SwinTransformerBase224V1,Adam,0.001,...,SwinTransformerBase224V1_Focal_W1,0.0,0.0,StepLR,0.5,5.0,NaN,NaN,224,224
2,42,30,MaskSplitByProfileDataset,BaseAugmentation,MaskDataLoader,512,1000,EfficientNetB0MultiHead,Adam,0.001,...,EfficientNetB0MultiHead_MixUp_1,0.0,0.0,StepLR,0.5,5.0,NaN,1.0,224,224
4,42,30,MaskSplitByProfileDataset,BaseAugmentation,MaskDataLoader,512,1000,EfficientViTB3MultiHead,Adam,0.001,...,EfficientViTB3MultiHeadV2,0.0,0.0,StepLR,0.5,5.0,NaN,NaN,128,96
6,42,30,MaskSplitByProfileDataset,BaseAugmentation,MaskDataLoader,512,1000,SwinTransformerBase224V1,Adam,0.001,...,fix_SwinTransformerBase224V1_focal_W1,0.0,0.0,StepLR,0.5,5.0,NaN,NaN,224,224
8,42,30,MaskSplitByProfileDataset,BaseAugmentation,MaskDataLoader,512,1000,FocalNetTinySRFMultiHead,Adam,0.001,...,FocalNetTinySRFMultiHeadV2,0.0,0.0,StepLR,0.5,5.0,NaN,NaN,128,96
9,42,30,MaskSplitByProfileDataset,CustomAugmentation,MaskDataLoader,256,1000,SwinTransformerBase224V1,Adam,0.001,...,fix_SwinTransformerV1_f1,0.0,0.0,StepLR,0.5,5.0,NaN,0.0,224,224
10,42,30,MaskSplitByProfileDataset,BaseAugmentation,MaskDataLoader,512,1000,SwinTransformerBase224V1,Adam,0.001,...,SwinTransformerBase224V1_focal_W2,0.0,0.0,StepLR,0.5,5.0,NaN,NaN,224,224
11,42,30,MaskSplitByProfileDataset,BaseAugmentation,MaskDataLoader,512,1000,SwinTransformerBase224V1,Adam,0.001,...,SwinTransformerBase224_best_loss,0.0,0.0,StepLR,0.5,5.0,NaN,NaN,224,224
13,42,30,MaskSplitByProfileDataset,BaseAugmentation,MaskDataLoader,512,1000,SwinTransformerBase224V3,Adam,0.001,...,SwinTransformerV3_3stage,0.0,0.0,StepLR,0.5,5.0,NaN,0.0,224,224
14,42,30,MaskSplitByProfileDataset,BaseAugmentation,MaskDataLoader,512,1000,SwinTransformerBase224V2,Adam,0.001,...,SwinTransformerV2_2stage,0.0,0.0,StepLR,0.5,5.0,NaN,0.0,224,224


In [188]:
rows = df[df["early_stopping"].isna() == True].index.values
df.loc[rows, "early_stopping"] = 0
df.loc[rows, "early_stopping"]

0     0.0
2     0.0
4     0.0
6     0.0
8     0.0
9     0.0
10    0.0
11    0.0
13    0.0
14    0.0
16    0.0
19    0.0
22    0.0
Name: early_stopping, dtype: float64

#### mixup

전부 사용안 한 것으로 처리 -> 0으로 변경

In [189]:
df[df["mixup"].isna() == True]

,seed,epochs,dataset,augmentation,dataloader,batch_size,valid_batch_size,model,optimizer,lr,...,file_name,kfold,cutmix,scheduler,lr_decay_rate,patience,early_stopping,mixup,resize_height,resize_width
0,42,30,MixUpSplitByProfileDataset,BaseAugmentation,MaskDataLoader,512,1000,SwinTransformerBase224V1,Adam,0.001,...,SwinTransformerBase224V1_Focal_W1,0.0,0.0,StepLR,0.5,5.0,0.0,NaN,224,224
1,42,15,MaskSplitByProfileDataset,CustomAugmentation,MaskDataLoader,256,45,SwinTransformerBase224V1,Adam,0.001,...,test_SwinTransformerV1_f1_fold0,1.0,0.0,StepLR,0.5,5.0,5.0,NaN,224,224
3,42,30,MaskSplitByProfileDataset,CustomAugmentation,MaskDataLoader,256,90,SwinTransformerBase224V1,Adam,0.001,...,final_SwinTransformerV1_f1_fold1,1.0,0.0,StepLR,0.5,5.0,5.0,NaN,224,224
4,42,30,MaskSplitByProfileDataset,BaseAugmentation,MaskDataLoader,512,1000,EfficientViTB3MultiHead,Adam,0.001,...,EfficientViTB3MultiHeadV2,0.0,0.0,StepLR,0.5,5.0,0.0,NaN,128,96
5,42,100,MaskSplitByProfileDataset,CustomAugmentation,MaskDataLoader,256,90,SwinTransformerBase224V1,Adam,0.001,...,finale_SwinTransformerV1,0.0,0.0,StepLR,0.5,5.0,7.0,NaN,224,224
6,42,30,MaskSplitByProfileDataset,BaseAugmentation,MaskDataLoader,512,1000,SwinTransformerBase224V1,Adam,0.001,...,fix_SwinTransformerBase224V1_focal_W1,0.0,0.0,StepLR,0.5,5.0,0.0,NaN,224,224
7,42,15,MaskSplitByProfileDataset,CustomAugmentation,MaskDataLoader,256,45,SwinTransformerBase224V1,Adam,0.001,...,test_SwinTransformerV1_f1_fold2,1.0,0.0,StepLR,0.5,5.0,5.0,NaN,224,224
8,42,30,MaskSplitByProfileDataset,BaseAugmentation,MaskDataLoader,512,1000,FocalNetTinySRFMultiHead,Adam,0.001,...,FocalNetTinySRFMultiHeadV2,0.0,0.0,StepLR,0.5,5.0,0.0,NaN,128,96
10,42,30,MaskSplitByProfileDataset,BaseAugmentation,MaskDataLoader,512,1000,SwinTransformerBase224V1,Adam,0.001,...,SwinTransformerBase224V1_focal_W2,0.0,0.0,StepLR,0.5,5.0,0.0,NaN,224,224
11,42,30,MaskSplitByProfileDataset,BaseAugmentation,MaskDataLoader,512,1000,SwinTransformerBase224V1,Adam,0.001,...,SwinTransformerBase224_best_loss,0.0,0.0,StepLR,0.5,5.0,0.0,NaN,224,224


In [190]:
rows = df[df["mixup"].isna() == True].index.values
df.loc[rows, "mixup"] = 0
df.loc[rows, "mixup"]

0     0.0
1     0.0
3     0.0
4     0.0
5     0.0
6     0.0
7     0.0
8     0.0
10    0.0
11    0.0
12    0.0
15    0.0
16    0.0
17    0.0
18    0.0
19    0.0
20    0.0
21    0.0
23    0.0
24    0.0
25    0.0
Name: mixup, dtype: float64

#### 최종 확인

In [192]:
df.isna().sum().sum()

0

In [193]:
df.to_csv('reports.csv', index=False)

In [195]:
new_df = pd.read_csv("./reports.csv")
new_df.isna().sum().sum()

0